# Backtracking Poisson image deblurring

we consider the problem of image deconvolution for sparse images corrupted by Poisson noise with undersampling. This problem can be encountered, for instance, in super-resolution microscopy \cite{}. Given a non-negative image $z\in R^n$, it can be formulated as the inverse problem of retrieving a sparse and non-negative $x\in R^n$ from $z=\mathcal{P}(MHx+b)$, where $M:R^n\rightarrow R^m$ is a subsampling operator (note that $m<n$), $H\in R^{n\times n}$ is a blurring operator computed for a given point spread function (PSF), $b\in R^m$ stands for a positive background term and $\mathcal{P}(w)$ denotes a realisation of a Poisson-distributed $n$-dimensional random vector with parameter $w\in R^n$. 

As it is well-known, whenever Poisson noise is observed in the data, a statistically consistent data fidelity term is the generalised Kullback-Leibler divergence functional defined by:
$$
    KL(MHx+b;z):=\sum_{i=1}^n\left(z_i\log \frac{z_i}{(MHx)_i+b_i}+(MHx)_i+b_i-z_i\right),
$$
and where  the convention $0\log 0=0$ is adopted. By enforcing sparsity using a standard $\ell^1$ penalty and denoting by $\iota_{\geq 0}(\cdot)$ the indicator function on the non-negative orthant, we consider:
$$
\min_{x\in R^n}F(x):=\underbrace{KL(MHx+b,z)}_{:=f(x)}+\underbrace{\lambda\|x\|_1+i_{\geq 0}(x)}_{:=h(x)}.
$$

### Importing libraries, generating data and operators

In [ ]:
#Importing libraries
import numpy as np
import scipy as scp
import pylab as pyl
import matplotlib.pyplot as plt
import scipy
import scipy.io
from PIL import Image
import time
import holoviews as hv
import panel as pn
from scipy import signal
hv.extension('bokeh')

In [ ]:
#Importing additional packages: algorithms.py contains the algorithms to apply and visualizer.py is useful to plot loss functions.
import os.path
if not os.path.isfile('./visualizer.py'):
    !wget https://github.com/HippolyteLBRRR/Composite_optimization/raw/main/visualizer.py
if not os.path.isfile('./algorithms.py'):
    !wget https://github.com/HippolyteLBRRR/Composite_optimization/raw/main/algorithms.py
from algorithms import *
from visualizer import *

In [ ]:
# Display setting
options0=dict(width=400,height=400,xaxis=None,yaxis=None,toolbar=None,cmap='hot')

In [ ]:
# Importing data
pth_kernel='./ISBI_1.mat'
data = scipy.io.loadmat(pth_kernel)
print(data)
im=data["gt"]
kern=data["PSF"]
M=data["M"]
bg=data["b"]
im_det=data["Acq"]+bg
pn.Row(hv.Image(im).opts(**options0),hv.Image(im_det).opts(**options0),hv.Image(kern).opts(**options0))

In [ ]:
#Definition of the operators

def Convolution(x, A):
    # A is the fft of the kernel!
    y = np.real(np.fft.ifft2(A * np.fft.fft2(x)))
    return y

def Convolution_adj(x,A):
    # A is the fft of the kernel!
    y = np.real(np.fft.ifft2(np.conj(A) * np.fft.fft2(x)))
    return y

def UnderSampling(x,M):
    return M@x@M.T

def Poisson(x):
    return np.random.poisson(x)

def KL1(H,M,x,z,bg):
    MHx=UnderSampling(H(x),M)
    temp=(z!=0)*(z*np.log(z/(MHx+bg)))+MHx+bg-z
    return np.sum(temp)

def Grad_KL1(H,HT,M,x,z,bg):
    MHx=UnderSampling(H(x),M)
    return HT(UnderSampling(np.ones(MHx.shape)-z/(MHx+bg),M.T))

def operators_poisson(kern,M,lmbd,im,bg):
    #kern : kernel
    #lmbd : regularization parameter
    #im : damaged image
    #bg : background value
    N=np.max(np.shape(M))
    n=np.min(np.shape(M))
    PSF = np.fft.fft2(np.fft.fftshift(kern), [N,N])
    Convo = lambda x: Convolution(x,PSF)
    ConvoT = lambda x: Convolution_adj(x,PSF)
    f = lambda x: KL1(Convo,M,x,im,bg)
    Df = lambda x: Grad_KL1(Convo,ConvoT,M,x,im,bg)     
    h = lambda x: lmbd*np.sum(np.abs(x))
    proxh = lambda x,s: (x-lmbd*s)*(x>lmbd*s)
    F = lambda x: f(x)+h(x)
    im_init=Convo(UnderSampling(im,M.T))
    return F,f,h,Df,proxh,im_init

In [ ]:
# Computing the operators for a given set of parameters
lmbd=8
F,f,h,Df,proxh,im_init=operators_poisson(kern,M,lmbd,im_det,bg)
exit_crit = lambda x,y: npl.norm(np.ravel(x-y),2) # exit criteria
sp = lambda x,y: np.dot(np.ravel(x),np.ravel(y)) # scalar product considered
hv.Image(im_init).opts(**options0)

### Applying first-order methods to this problem

In [ ]:
# Documentation for the algorithms:
# help(FISTA_BT)
# help(Free_FISTA)

In [ ]:
# Defining the parameters of resolution
Niter=1000 # Maximum number of iterations
L0=100 # Initial estimation of the Lipschitz constant of nabla f (arbitrary)
epsilon=1e-6 # Expected accuracy for the composite gradient

In [ ]:
# FISTA with backtracking
rho=0.8
delta=0.95
xBT,cout_BT,ctime_BT,Lk=FISTA_BT(im_init, L0, rho, delta, Niter, epsilon, f, Df, proxh, h, exit_crit, sp,out_L=True,exit_norm=True,track_ctime=True)
Plot_BT=To_Plot(cout_BT,"FISTA with backtracking, rho=%f, delta=%f, L0=%i"%(rho,delta,L0),ctime_BT)

In [ ]:
# Free FISTA
rho=0.8
delta=0.95
xRBT,cout_BTR,ctime_BTR,tab_L,tab_mu=Free_FISTA(im_init, L0, rho, delta, Niter, epsilon, f, h, Df, proxh,exit_crit=exit_crit, sp=sp, out_cost=True,out_L=True,exit_norm=True,out_condition=True,track_ctime=True)
Plot_BTR=To_Plot(cout_BTR,"Free-FISTA, rho=%f, delta=%f, L0=%i"%(rho,delta,L0),ctime_BTR)

In [ ]:
Plot([Plot_BT,Plot_BTR],ite=False,eps=1e-5,fontsize = 25)
plt.xlabel("Computation time (in seconds)",fontsize=25)
plt.ylabel("$\log(F(x_k)-\hat F)$",fontsize=25)

In [ ]:
# Displaying the resulting approximation of the solution
hv.Image(xBT).opts(**options0)
